In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_dir = "../input/train";
test_dir = "../input/test";

In [ ]:
train = pd.read_csv(train_dir + ".csv")
train = train[train.Id != "new_whale"]

In [ ]:
#import cv2 as cv

In [ ]:
#i = 0
#for x in X:
 #   img = cv.resize(cv.imread(train_dir + '/' + x),(100,100))
  #  cv.imwrite(x[:-4] + "_copy.jpg",img)
   # i += 1
    #if (i % 250) == 0:
     #   print(i)

In [ ]:
from keras.applications.resnet50 import ResNet50
model = ResNet50(include_top=False, weights = 'imagenet', input_shape = (100,100,3), pooling = 'avg')

In [ ]:
for layer in model.layers[:-5]:
    layer.trainable = False

In [ ]:
from keras import Model
from keras.layers import Dense
from keras import optimizers
x = model.output
x=Dense(5004,activation='softmax')(x)
model_final=Model(inputs=model.input,outputs=x)

In [ ]:
#model_final.summary()

In [ ]:
from keras import optimizers
model_final.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['acc'])


In [ ]:
model_final.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')


train_generator = train_datagen.flow_from_dataframe(
dataframe = train,
directory = "../input/train/",
x_col = "Image",
y_col = "Id",
target_size = (100,100),
class_mode = "categorical",
batch_size = 32    
)

In [ ]:
from keras.callbacks import EarlyStopping
history = model_final.fit_generator(train_generator,
      steps_per_epoch=train_generator.samples,
      epochs=1,
      verbose=1,
      callbacks=[EarlyStopping(monitor='acc', mode='max')])

In [ ]:
model_final.save_weights('model1weights.h5')
model_final.save('model1.h5')